Para resolver este ejercicio encontrarás adjunto una carpeta comprimida con una serie de tablas (lego-database.zip).

Imagina que tú eres el encargado del área de compras de LEGO y debes preparar la estrategia de compras para las campañas de este año.

Este año se tendrá especial énfasis en 3 temáticas con las siguientes expectativas de producción:
1. Star Wars: 5.000.000 unidades
2. Héroes: 5.000.000 unidades
3. Building: 5.000.000 unidades

La producción de estas nuevas unidades empieza en 1 mes y no se tienen piezas en inventario.

### Análisis de la Estructura de la Base de Datos

Según el esquema proporcionado, las tablas importantes para este análisis son:

- **`inventory_sets`**: Para obtener los inventarios relacionados con los sets.
- **`sets`**: Para identificar los sets por nombre y tema (como "Star Wars", "Héroes", "Building").
- **`themes`**: Para categorizar los temas de los sets.
- **`inventory_parts`**: Para identificar las piezas necesarias para los sets específicos.
- **`colors`**: Para obtener los colores de las piezas necesarias.
- **`parts`** y **`part_categories`**: Para obtener información adicional sobre los tipos y categorías de piezas.

<p align="center">
  <img src="./Data/downloads_schema.png" alt="LEGO Logo">
</p>

### Determinación de Cantidad y Proporciones de Colores para Producción

Para alcanzar una producción de 5,000,000 unidades para los temas "Star Wars", "Heroes" y "Building", se sigue un enfoque sistemático que involucra un análisis detallado de los datos disponibles. El proceso se desglosa de la siguiente manera:

#### Filtrado por Temática
Comenzamos con la tabla `themes` para identificar los temas de interés. Los temas permiten focalizar las áreas de producción y reducen el rango de interés a conjuntos específicos. En el caso de LEGO, los temas son cruciales para determinar qué sets se fabrican y cuál es el equipo responsable.

#### Relación con Sets
Usamos la información de `themes` para filtrar los sets asociados a cada tema. Esto se logra mediante la clave foránea `theme_id` en la tabla `sets`, que se vincula con `id` en la tabla `themes`.

#### Filtrado de Inventarios
Una vez obtenidos los sets relevantes, filtramos los inventarios asociados a esos sets utilizando la clave foránea `set_num` en la tabla `inventories`, que corresponde a `set_num` en `sets`.

#### Obtención de Piezas
Con los inventarios filtrados, conectamos con la tabla `inventory_parts` para obtener detalles sobre las piezas utilizadas en cada inventario. Esta tabla se une con `colors` a través de la clave `color_id` en `inventory_parts` y `id` en `colors`.

#### Cálculo de Proporciones de Colores
Finalmente, con la información de piezas y colores, calculamos la distribución de colores, ajustando las cantidades según la producción requerida. Esto implica calcular la proporción de cada color en base al total de piezas y escalar estas proporciones para cumplir con el objetivo de producción.

### Flujo de Trabajo Implementado

- **Ruta de Filtrado (amarillo)**: Incluye la selección de datos basados en los temas, sets, y inventarios específicos.
- **Ruta de Unión de Tablas (verde)**: Incluye la combinación de datos de diferentes tablas para obtener la distribución de colores.

Esta metodología asegura una comprensión clara de los requisitos de colores para la producción deseada, proporcionando una base sólida para planificar y ejecutar la fabricación de sets de LEGO.

In [79]:
import pandas as pd
import os

extract_dir = './Data'
# Cargar los archivos CSV en DataFrames
colors_df = pd.read_csv(os.path.join(extract_dir, 'colors.csv'))
inventories_df = pd.read_csv(os.path.join(extract_dir, 'inventories.csv'))
inventory_parts_df = pd.read_csv(os.path.join(extract_dir, 'inventory_parts.csv'))
inventory_sets_df = pd.read_csv(os.path.join(extract_dir, 'inventory_sets.csv'))
part_categories_df = pd.read_csv(os.path.join(extract_dir, 'part_categories.csv'))
parts_df = pd.read_csv(os.path.join(extract_dir, 'parts.csv'))
sets_df = pd.read_csv(os.path.join(extract_dir, 'sets.csv'))
themes_df = pd.read_csv(os.path.join(extract_dir, 'themes.csv'))

# Agregar el nombre del tema a los sets uniendo con themes
sets_with_themes_df = sets_df.merge(themes_df, left_on='theme_id', right_on='id', suffixes=('_set', '_theme'))
sets_with_themes_df.rename(columns={'name_theme': 'theme_name'}, inplace=True)

# Filtrar sets para las temáticas seleccionadas
selected_themes = ["Star Wars", "Heroes", "Building"]
sets_in_selected_themes_df = sets_with_themes_df[sets_with_themes_df['theme_name'].isin(selected_themes)]

# Definir la producción requerida por tema
production_requirements = {
    'Star Wars': 5000000,
    'Heroes': 5000000,
    'Building': 5000000
}

# Función para obtener la distribución de colores por tema
def calculate_color_distribution(theme_name, production_required):
    # Filtrar sets del tema específico
    theme_sets_df = sets_with_themes_df[sets_with_themes_df['theme_name'] == theme_name]
    
    # Unir sets con los inventarios y luego con las piezas para obtener las partes involucradas
    selected_inventories_df = inventories_df[inventories_df['set_num'].isin(theme_sets_df['set_num'])]
    selected_inventory_parts_df = inventory_parts_df[inventory_parts_df['inventory_id'].isin(selected_inventories_df['id'])]
    
    # Unir con la tabla de colores para obtener la distribución de colores
    color_distribution_df = selected_inventory_parts_df.merge(colors_df, left_on='color_id', right_on='id')

    # Obtenemos las partes del inventario de los sets del tema
    inventory_parts_filtered = color_distribution_df
    
    # Calcular la cantidad total de piezas en el tema
    total_pieces_theme = color_distribution_df['quantity'].sum()
    
    # Calcular las proporciones de piezas escaladas a la producción requerida
    color_distribution_df['scaled_quantity'] = (color_distribution_df['quantity'] / total_pieces_theme) * production_required
    
    # Agrupar por color para obtener el total por color
    color_proportions = color_distribution_df.groupby('name').agg({'quantity': 'sum', 'scaled_quantity': 'sum'}).reset_index()
    total_pieces_theme = color_distribution_df.groupby('part_num').agg({'quantity': 'sum', 'scaled_quantity': 'sum'}).reset_index()
    
    return color_proportions, total_pieces_theme

def get_parts_categories_count(total_pieces_theme):
    parts = parts_df[parts_df['part_num'].isin(total_pieces_theme['part_num'])]
    parts_categories = part_categories_df[part_categories_df['id'].isin(parts['part_cat_id'])]
    parts_categories = len(parts_categories['name'].unique())

    return parts_categories
# Obtener la distribución de colores para cada temática

star_wars_distribution,total_pieces_theme_star_wars = calculate_color_distribution('Star Wars', production_requirements['Star Wars'])
print('Distribución de colores para Star Wars')
star_wars_distribution




Distribución de colores para Star Wars


,name,quantity,scaled_quantity
0,Black,8935,1.237260e+06
1,Blue,834,1.154869e+05
2,Bright Green,9,1.246261e+03
3,Bright Light Orange,5,6.923673e+02
4,Bright Light Yellow,2,2.769469e+02
...,...,...,...
59,Unknown,1,1.384735e+02
60,Violet,35,4.846571e+03
61,White,2288,3.168273e+05
62,Yellow,338,4.680403e+04


In [80]:
parts_categories_star_wars = get_parts_categories_count(total_pieces_theme_star_wars)
print('Cantidad de categorías de piezas en Star Wars')
print(parts_categories_star_wars)

Cantidad de categorías de piezas en Star Wars
48


In [81]:
heroes_distribution, total_pieces_theme_heroes = calculate_color_distribution('Heroes', production_requirements['Heroes'])
print("Distribución de colores para Héroes:")
heroes_distribution

Distribución de colores para Héroes:


,name,quantity,scaled_quantity
0,Black,627,1.290655e+06
1,Blue,211,4.343351e+05
2,Bright Green,4,8.233841e+03
3,Bright Light Orange,12,2.470152e+04
4,Dark Bluish Gray,205,4.219844e+05
5,Dark Brown,4,8.233841e+03
6,Dark Green,4,8.233841e+03
7,Dark Purple,3,6.175381e+03
8,Dark Tan,11,2.264306e+04
9,Flat Silver,185,3.808152e+05


In [82]:
parts_categories_heroes = get_parts_categories_count(total_pieces_theme_heroes)
print('Cantidad de categorías de piezas en Heroes')
print(parts_categories_heroes)

Cantidad de categorías de piezas en Heroes
27


In [83]:
building_distribution,total_pieces_theme_building = calculate_color_distribution('Building', production_requirements['Building'])
print("Distribución de colores para Building:")
building_distribution

Distribución de colores para Building:


,name,quantity,scaled_quantity
0,Black,1558,428988.380417
1,Blue,1224,337022.963820
2,Bright Green,40,11013.822347
3,Bright Light Orange,6,1652.073352
4,Brown,11,3028.801145
5,Dark Blue,132,36345.613745
6,Dark Bluish Gray,1120,308387.025717
7,Dark Brown,15,4130.183380
8,Dark Gray,10,2753.455587
9,Dark Green,12,3304.146704


In [84]:
parts_categories_building = get_parts_categories_count(total_pieces_theme_building)
print('Cantidad de categorías de piezas en Heroes')
print(parts_categories_building)

Cantidad de categorías de piezas en Heroes
39
